# Descriptive Statistics and Tests - Introduction


## Test for Stationarity: Augmented Dickey-Fuller Test

Performs a test in the form of null hypothesis and returns a p value.

- Null hypothesis: $\phi=1$

- If p value below set alpha (0.05), then reject the null hypothesis, i.e., the time series is considered stationary.

- If p value is high, then we fail to reject and time series can be considered non-stationary

## Granger Causality Test

Also a hypothesis test. Determines if one time series is useful in forecasting another.

Correlation between time series is easy to measure (one goes up, other goes up). One time series changes causes another time series to change in the future is a completely different beast. Granger Causality can help here. However, even if there is strong causality reported, there could still be a third party factor that has causality on both, so it is in reality some outside factor and not the time series that affects the other time series! What can help here is subject matter expertise.

## AIC Akaike Information Criterion

AIC evaluates a collection of models and estimates the quality of each model relative to the others. Penalties are provided for the number of parameters used in an effort to thwart overfitting.

Rather have a simpler model that performs slightly less over a more complex model that is only miniscule better than the simpler model.

## BIC Bayesian Information Criterion

Similar to AIC but uses Bayesian mathematics.